In [2]:
import matplotlib.pyplot as plt
import torch
from torch.quasirandom import SobolEngine

import numpy as np
from alse.accuracy import get_accuracy
from alse.utils import identify_samples_which_satisfy_constraints, normalize, unnormalize
from mpl_toolkits import mplot3d
# from matplotlib import cm
from alse.utils import read_excel
from alse.alse import alse
import copy
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}

X, width, pow_cap, wth = read_excel("../test_data/trial_NaN.xlsx",["P (W)", "V (mm/min)"], ["widths avg (mm)", "powder capt %", "wth"])
bounds = torch.tensor([[900, 600], [2700, 1800]])
constraints = [("gt", 2.5), ("gt", 0.65), ("gt", 6)]

algo = alse(X, bounds, [width, pow_cap, wth], constraints)

algo.initialize_model(["reg", "reg", "reg"])

algo.next_test_points(5)

/opt/conda/lib/python3.9/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:124: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2189.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution
/opt/conda/lib/python3.9/site-packages/gpytorch/distributions/multivariate_normal.py:259: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/gpytorch/distributions/multivariate_normal.py:259: NumericalWarning: Negative variance values detected. Thi

tensor([[2700.0000,  869.4183],
        [2347.1309,  679.9690],
        [2379.8477,  979.5935],
        [2315.4209, 1218.6531],
        [2633.7876, 1138.5745]])